In [ ]:
from pymongo.mongo_client import MongoClient

uri = "mongodb+srv://<user>:<password>@espi.bebf5dd.mongodb.net/?retryWrites=true&w=majority&appName=Espi"

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [2]:
db=client["mytestdb"]
collection=db["mytestcollection"]

In [111]:
collection.insert_one({"name": "3 Idiots",
 "genres": "Comedy",
 "rating": 8.4,
 "content": "Two friends search for their lost companion and revisit their college days, revealing a bet, a wedding, and secrets."})

In [7]:
from docx import Document as DocxDocument
from docx.opc.constants import RELATIONSHIP_TYPE as RT
import glob
from langchain.schema import Document as LC_Document

links=[]

def load_docs_one_per_file(folder_path):
    docs=[]
    for path in glob.glob(folder_path+"/*.docx"):
        docx=DocxDocument(path)

        rels={
            rel.rId: rel.target_ref
            for rel in docx.part.rels.values()
            if rel.reltype==RT.HYPERLINK
        }

        full_md=[]
        for para in docx.paragraphs:
            md_text=""
            p=para._p
            for element in p:
                if element.tag.endswith('}hyperlink'):
                    rid=element.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id')
                    url=rels.get(rid)
                    display=''.join(node.text or '' for node in element.iter() if node.tag.endswith('}t'))
                    md_text+=f"[{display}]({url}) " if url else display
                    links.append(md_text)
                elif element.tag.endswith('}r'):
                    run_text=''.join(node.text or '' for node in element.iter() if node.tag.endswith('}t'))
                    md_text+=run_text

            if md_text.strip():
                full_md.append(md_text)

        joined="\n\n".join(full_md)
        docs.append(LC_Document(page_content=joined,
                                metadata={"source": path}))
    return docs

all_docs=load_docs_one_per_file("Documents")


In [2]:
links

[' [Training for AI Engineers](https://docs.google.com/document/d/1NfmEDyxrJ7Tz7Wq4lAJHx1fQ4bPpM5v07dPTM4pjOsM/edit?usp=sharing) ',
 '[Training of AI Designer](https://docs.google.com/document/d/1GkGijGY1jLPrmm-MSRYoCj3FTJwWB_RxIqapRMF6Eg0/edit?usp=sharing) ',
 ' [Training for AI Engineers](https://docs.google.com/document/d/1NfmEDyxrJ7Tz7Wq4lAJHx1fQ4bPpM5v07dPTM4pjOsM/edit?usp=sharing) ',
 '[Training of AI Designer](https://docs.google.com/document/d/1GkGijGY1jLPrmm-MSRYoCj3FTJwWB_RxIqapRMF6Eg0/edit?usp=sharing) ',
 'Designers:  Join Pitch Day & Team Match (within 24 fill out AI products interested in on [Team Match.xls](https://docs.google.com/spreadsheets/d/1WFSjvQTxtTfs0Hyocgj_0QNirXswTFMf8hsgZTRJn5E/edit?usp=sharing) ',
 'Engineers join Zoom Pitch Day and fill out [Google Ranking form](https://forms.gle/VE2DCuHywse7KCW29) ',
 '[Training for Engineers ](https://docs.google.com/document/d/1NfmEDyxrJ7Tz7Wq4lAJHx1fQ4bPpM5v07dPTM4pjOsM/edit?usp=sharing) ',
 '[Training for Designers](ht

In [ ]:
import re
def extract_youtube_links(link_list: list[str]) -> list[dict]:
    yt_links=[]
    pattern=re.compile(r"\[([^\]]+)\]\((https?://(?:www\.)?(?:youtube\.com|youtu\.be)[^\)]+)\)")
    for md in link_list:
        match=pattern.search(md)
        if match:
            title=match.group(1).strip()
            url=match.group(2)
            yt_links.append({"title": title, "url": url})
    return yt_links


In [ ]:
def get_youtube_id(url: str) -> str | None:
    
    pattern=re.compile(r"(?:youtube\.com/watch\?v=|youtu\.be/)([^&#]+)")
    match=pattern.search(url)
    return match.group(1) if match else None


In [6]:
import os
youtube_contents=[]
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, VideoUnavailable, VideoUnplayable
youtube_links=extract_youtube_links(links)
count=0
print(len(youtube_links))
for yt in youtube_links:
    id=get_youtube_id(yt['url'])
    if id:
        count+=1
        try:
            transcript_list=YouTubeTranscriptApi.get_transcript(id, languages=['en'])
            transcript=" ".join([item['text'] for item in transcript_list])
            youtube_contents.append(transcript)

        except (VideoUnavailable, TranscriptsDisabled, VideoUnplayable) as e:
            print(f"Skipping video {id}: {e}")
        print(f"Title: {yt['title']}, URL: {yt['url']}, ID: {id}")
print(count)


71


ConnectionError: HTTPSConnectionPool(host='www.youtube.com', port=443): Max retries exceeded with url: /watch?v=0qG_0CPQhpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002462DB666E0>: Failed to resolve 'www.youtube.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200 
)

chunks=splitter.split_documents(all_docs)

In [22]:
len(chunks)

54

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


In [ ]:
GOOGLE_API_KEY=os.environ['GOOGLE_API_KEY']

In [5]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY)

In [6]:
chat_template=PromptTemplate(
    template='''Extract important keywords from the text given below so that the context of the text can be understood.
                Only provide the points in comma separated format.
                \n\n
                Text : \n{text}
            ''',
    input_variables=["text"]
)

In [22]:
parser=StrOutputParser()

In [102]:
chain=chat_template|llm|parser

In [103]:
for i in range(49, len(chunks)):
    chunks[i].metadata['keywords']=chain.invoke({"text": chunks[i].page_content})
    print(i)

49
50
51
52
53


In [104]:
chunks[53].metadata

{'source': 'Documents\\Training For AI Engineer Interns.docx',
 'keywords': 'Python Fundamentals, Data Manipulation, Pandas, NumPy, Connecting Python to Databases, SQLAlchemy, psycopg2, SQL database'}

In [106]:
documents=[]
for i in range(len(chunks)):
    documents.append({"content": chunks[i].page_content,
                      "source":chunks[i].metadata['source'],
                      "keywords":chunks[i].metadata['keywords']})

In [112]:
collection.insert_many(documents)

In [7]:
PINECONE_API_KEY=os.environ['PINECONE_API_KEY']

In [ ]:
from pinecone import Pinecone

pc=Pinecone(api_key=PINECONE_API_KEY)
index=pc.Index("rag-chat-bot")

c:\Users\suman\OneDrive\Desktop\RAG_Chatbot_PMA\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from sentence_transformers import SentenceTransformer
embedding_model=SentenceTransformer('thenlper/gte-large')

In [9]:
def get_result(query, similar_result=5):
    embedding=embedding_model.encode(query)
    embedding=embedding.tolist()

    result=index.query(
        vector=embedding,
        top_k=similar_result,
        include_metadata=True,
        include_values=False
    )

    return result

In [69]:
query="What are the resources to learn about bias and variance of Machine Learning?"

In [70]:
result=get_result(query)

In [71]:
result

{'matches': [{'id': '68700534bc7cf06ff58aa9ab',
              'score': 0.871542454,
              'values': []},
             {'id': '68700534bc7cf06ff58aa9ac',
              'score': 0.84136349,
              'values': []},
             {'id': '68700534bc7cf06ff58aa982',
              'score': 0.837415814,
              'values': []},
             {'id': '68700534bc7cf06ff58aa9ad',
              'score': 0.833752096,
              'values': []},
             {'id': '68700534bc7cf06ff58aa983',
              'score': 0.832091451,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

In [72]:
from bson.objectid import ObjectId

In [ ]:
mylist=[]
for i in range(len(result['matches'])):
    value=result['matches'][i]['id']
    mylist.append(collection.find_one({"_id": ObjectId(value)}))

In [74]:
mylist

[{'_id': ObjectId('68700534bc7cf06ff58aa9ab'),
  'content': '[Docker for DataScientists ](https://www.youtube.com/watch?v=0qG_0CPQhpg) \n\nAdvanced AI Concepts\n\n[Machine Learning Master](https://machinelearningmastery.com/)  = Great source for complex concepts\n\nOverfitting and underfitting \n\n[Underfitting & Overfitting - Explained](https://youtu.be/o3DztvnfAJg) \n\n[Machine Learning Fundamentals: Bias and Variance](https://youtu.be/EuBBz3bI-aA) \n\nMLOps [What is MLOps?](https://youtu.be/OejCJL2EC3k) \n\n[MLOPS ZoomCamp ](https://github.com/DataTalksClub/mlops-zoomcamp) : 9-Week Course on Productionizing ML Services\n\nFeature Engineering [What is Feature Engineering?](https://youtu.be/AnZWCB1gpfE) \n\nHyperparameter Tunning [Parameters vs hyperparameters in machine learning](https://youtu.be/V4AcLJ2cgmU) \n\nModel Deployment [Machine Learning Model Deployment Explained | All About ML Model Deployment](https://youtu.be/SHyFjJ-tIJE) \n\nFraming the AI task',
  'source': 'Documents

In [75]:
combined_information=""
for i in range(len(mylist)):
    content=mylist[i]["content"]
    keywords=mylist[i]["keywords"]
    combined_information+=f"Content: {content},\nKeywords: {keywords}\n"

In [76]:
print(combined_information)

Content: [Docker for DataScientists ](https://www.youtube.com/watch?v=0qG_0CPQhpg) 

Advanced AI Concepts

[Machine Learning Master](https://machinelearningmastery.com/)  = Great source for complex concepts

Overfitting and underfitting 

[Underfitting & Overfitting - Explained](https://youtu.be/o3DztvnfAJg) 

[Machine Learning Fundamentals: Bias and Variance](https://youtu.be/EuBBz3bI-aA) 

MLOps [What is MLOps?](https://youtu.be/OejCJL2EC3k) 

[MLOPS ZoomCamp ](https://github.com/DataTalksClub/mlops-zoomcamp) : 9-Week Course on Productionizing ML Services

Feature Engineering [What is Feature Engineering?](https://youtu.be/AnZWCB1gpfE) 

Hyperparameter Tunning [Parameters vs hyperparameters in machine learning](https://youtu.be/V4AcLJ2cgmU) 

Model Deployment [Machine Learning Model Deployment Explained | All About ML Model Deployment](https://youtu.be/SHyFjJ-tIJE) 

Framing the AI task,
Keywords: Docker, Data Science, AI, Machine Learning, Overfitting, Underfitting, Bias, Variance, 